In [1]:
import os

from PIL import Image

import tools.dataset_tools as dataset_tools
from tools.frame_pb2 import Frame
import tools.plot_tools as plot_tools

In [2]:
!pip install natsort

In [3]:
from os import listdir
from os.path import isfile, join
from natsort import natsorted, ns
from PIL import Image
import numpy as np

def get_frame_pcl(frame_path):
    frame = dataset_tools.read_frame(frame_path)
    lidar = frame.lidars[0]
    return dataset_tools.decode_lidar(lidar)


In [4]:
def from_raw_to_bird_eye(pcl, frame, side_range = [-15, 15], fwd_range = [-0, 30]):
    '''
    # LIMIT VIEWING RANGE - To within a desired rectangle
    side_range = [-15, 15] # 10 metres on either side
    fwd_range = [-0, 30]   # 30 metres in front
    '''
    x_lidar = pcl[:, 0]
    y_lidar = pcl[:, 1]
    z_lidar = pcl[:, 2]
    # INDICES FILTER - of values within the desired rectangle
    # Note left side is positive y axis in LIDAR coordinates
    ff = np.logical_and((x_lidar > fwd_range[0]), (x_lidar < fwd_range[1]))
    ss = np.logical_and((y_lidar > -side_range[1]), (y_lidar < -side_range[0]))
    indices = np.argwhere(np.logical_and(ff, ss)).flatten()
    # POINTS TO USE FOR IMAGE
    x_img = -y_lidar[indices]       # x axis is -y in LIDAR
    y_img = x_lidar[indices]        # y axis is x in LIDAR
    pixel_values = z_lidar[indices] # Height values used for pixel intensity

    # Shift values so (0,0) is the minimum value
    x_img -= side_range[0]
    y_img -= fwd_range[0]
    
    # PLOT THE IMAGE
    cmap = "jet"    # Color map to use
    dpi = 100       # Image resolution
    x_max = side_range[1] - side_range[0]
    y_max = fwd_range[1] - fwd_range[0]
    fig, ax = plt.subplots(figsize=(600/dpi, 600/dpi), dpi=dpi)
    ax.scatter(x_img, y_img, s=1, c=pixel_values, linewidths=0, alpha=1, cmap=cmap)
    ax.set_facecolor((0, 0, 0))  # Set regions with no points to black
    ax.axis('scaled')  # {equal, scaled}
    ax.xaxis.set_visible(False)  # Do not draw axis tick marks
    ax.yaxis.set_visible(False)  # Do not draw axis tick marks
    plt.xlim([0, x_max])  # prevent drawing empty space outside of horizontal FOV
    plt.ylim([0, y_max])  # prevent drawing empty space outside of vertical FOV
    fig.savefig(f"bev_data/{frame}.jpg", dpi=dpi, bbox_inches='tight', pad_inches=0.0)

    return (x_img, y_img, pixel_values)


In [ ]:
mypath = "./data/"
onlyfiles = natsorted([mypath+f for f in listdir(mypath) if isfile(join(mypath, f))], key=lambda y: y.lower())

for f in onlyfiles:
    print('frame: ', f)
    pcl = get_frame_pcl(f)
    bev = from_raw_to_bird_eye(pcl,f.replace("./data/","").replace(".pb",""))